#**Main Application**

In [ ]:
!pip install streamlit
!pip install --upgrade boto3
!pip install PyMuPDF python-docx pandas openpyxl doc2docx pyngrok
access_key = "Insert Key Here"
access_secret = "Insert Secret Here"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 

In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.171.16.37


In [ ]:
import streamlit as st
from pyngrok import ngrok
import os

# Set your ngrok authtoken here
NGROK_AUTH_TOKEN = "Insert AuthToken Here"

def run_streamlit_with_ngrok():
    # Set ngrok authtoken
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    # Start ngrok tunnel
    try:
        public_url = ngrok.connect(addr="8501", proto="http", name="streamlit")
        print(f"Streamlit app is running on: {public_url}")
    except Exception as e:
        print(f"Error setting up ngrok: {e}")
        return

    # Run Streamlit app
    os.system("streamlit run main.py &")

if __name__ == "__main__":
    run_streamlit_with_ngrok()

Streamlit app is running on: NgrokTunnel: "https://7012-34-148-200-43.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
import streamlit as st
import boto3
import json
import re
import time
import fitz
from botocore.exceptions import ClientError

# API credentials
access_key = "Insert Key Here"
access_secret = "Insert Secret Here"

cleaned_content = ""

files = {
    "example_patients": "/content/aphasiapatient - ENGLISH - aphasiapatient (1).pdf",
}

# Function to read PDF files
def read_pdf(file_path):
    document = fitz.open(file_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

example_patients = read_pdf(files["example_patients"])

def preprocess_transcript(file_content):
    # Split the content into lines
    lines = file_content.split('\n')

    # Keep only lines starting with '*INV:' or '*PAR:', and replace the asterisk with the label
    cleaned_lines = []
    for line in lines:
        line = line.strip()
        if line.startswith('*INV:'):
            cleaned_lines.append('INV: ' + line[5:].strip())
        elif line.startswith('*PAR:'):
            cleaned_lines.append('PAR: ' + line[5:].strip())

    # Join the cleaned lines back into a single string
    cleaned_content = '\n'.join(cleaned_lines)

    return cleaned_content
def clean_transcript(input_text):
    # Keep only lines with speaker labels (*INV:, *PAR:) and their content
    cleaned_lines = re.findall(r'^\*[A-Z]+:.*$', input_text, re.MULTILINE)
    # Remove various artifacts and clean up the lines
    cleaned_lines = [re.sub(r'\d+_\d+$|•\d+_\d+•|\[.*?\]|&=.*?(?:\s|$)|&-\w+|\+<|\(\.\.\)|\.\.\.|‡', '', line) for line in cleaned_lines]
    # Remove leading/trailing whitespace and any remaining punctuation at the end
    cleaned_lines = [re.sub(r'^\s+|\s+$|[,\.!?]+$', '', line) for line in cleaned_lines]
    # Remove empty lines and lines with only punctuation/spaces
    cleaned_lines = [line for line in cleaned_lines if re.search(r'[a-zA-Z]', line)]
    return '\n'.join(cleaned_lines)

# Function to call Bedrock API with retry logic
def call_bedrock(prompt, max_retries=3):
    retries = 0
    while retries <= max_retries:
        try:
            bedrock = boto3.client(service_name='bedrock-runtime',
                                   region_name='us-east-1',
                                   aws_access_key_id=access_key,
                                   aws_secret_access_key=access_secret)

            body = json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 4096,
                "messages": [
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": prompt
                            }
                        ]
                    }
                ],
                "temperature": 0.1,
                "top_p": 1
            })

            modelId = 'anthropic.claude-3-5-sonnet-20240620-v1:0'
            accept = 'application/json'
            contentType = 'application/json'

            response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
            response_body = json.loads(response.get('body').read())
            text = response_body['content'][0]['text']
            return text
        except ClientError as e:
            if e.response['Error']['Code'] == 'ThrottlingException':
                if retries < max_retries:
                    wait_time = 120  # 2 minutes
                    st.warning(f"Rate limit reached. Waiting for {wait_time} seconds before retrying...")
                    time.sleep(wait_time)
                    retries += 1
                else:
                    raise Exception("Max retries reached. Unable to process request.")
            else:
                raise e
        except Exception as e:
            raise Exception(f"Error in call_bedrock: {str(e)}")


# Streamlit UI
st.set_page_config(page_title="Aphasia Analysis", page_icon=":brain:")

st.title("Aphasia Type and WAB Score Analysis")

uploaded_files = st.file_uploader("Choose files", type=["txt", "rtf", "cha"], accept_multiple_files=True)

instructions = f"""
The factors of Aphasic speech that I need counted are. Provide the number of occurrences for each patient. Make sure you only look at the lines that start with @PAR:

1. Non-fluent speech: Slow, halting speech with frequent pauses
2. Word retrieval issues: Use of fillers, circumlocutions, semantic paraphasias
3. Grammatical errors: Incorrect function words, verb tenses, simplified sentences
4. Repetitions and revisions: Repeating words/phrases due to word-finding difficulties
5. Paraphasias: Unintended word substitutions (semantic or phonemic)
6. Neologisms: Creation of non-existent words
7. Perseveration: Unintentional repetition of words/phrases
8. Comprehension issues: Difficulty understanding complex sentences or fast speech


Summary of Evaluation Results:
Write a description for each of these factors in the patient.

Fluency: Assessment of the patient’s fluency, including spontaneous speech, language sample analysis, and subtest scores.
Comprehension: Evaluation of auditory comprehension, word recognition, and the ability to follow commands.
Naming: Results from naming tests, including spontaneous naming, phonological errors, and performance on naming tasks.
Semantic Processing: Evaluation of the patient’s ability to process semantic information.
Repetition: Assessment of the ability to repeat words, phrases, and non-words.

Fluency: (insert score here; scale of 0-10) Remember that this can be below 5
Comprehension: (insert score here; scale of 0-10) Remember that this can be below 5
Repetition: (insert score here; scale of 0-10)Remember that this can be below 5
Naming: (insert score here; scale of 0-10) Remember that this can be below 5

When I refer to the spreadsheet, use the {example_patients} file to compare the transcript I am giving you. This is a file of aphasic patients of all types, ages, genders, and range of severity. Based on your comparison to these patients, determine the aphasia type and WAB score.

Broca's Aphasia:
- Spontaneous speech: Non-fluent, effortful, telegraphic
- Auditory comprehension: Relatively preserved, especially for simple material
- Repetition: Impaired
- Naming: Difficult, with frequent pauses
- Reading: Often preserved for single words, difficulty with sentences
- Writing: Severely impaired, similar to speech output

Typical errors: Agrammatism, word-finding difficulties, articulation problems

Associated brain region: Posterior inferior frontal gyrus (Broca's area)

Case example: A 55-year-old right-handed man presents with halting, effortful speech following a left frontal lobe stroke. He can understand simple commands but struggles to produce complete sentences, often omitting function words and inflections.

Wernicke's Aphasia:
- Spontaneous speech: Fluent, often with excessive output
- Auditory comprehension: Severely impaired
- Repetition: Impaired
- Naming: Impaired, with semantic paraphasias
- Reading: Often impaired, especially for comprehension
- Writing: Impaired, with semantic errors and jargon

Typical errors: Semantic paraphasias, neologisms, jargon

Associated brain region: Posterior superior temporal gyrus (Wernicke's area)

Case example: A 70-year-old woman presents with fluent but largely incomprehensible speech following a left temporal lobe stroke. She speaks at length but with little meaningful content, and has significant difficulty understanding even simple commands.

Conduction Aphasia:
- Spontaneous speech: Fluent with phonemic paraphasias
- Auditory comprehension: Relatively preserved
- Repetition: Severely impaired
- Naming: Mildly impaired, with phonemic errors
- Reading: Often preserved, especially for comprehension
- Writing: Impaired, with phonemic errors

Typical errors: Frequent phonemic paraphasias, repetition attempts and self-corrections

Associated brain region: Arcuate fasciculus or left supramarginal gyrus

Case example: A 62-year-old woman presents with fluent speech but significant difficulty repeating words and phrases. She understands spoken language well but makes frequent sound-based errors in her speech, often attempting to correct herself.

Global Aphasia:
- Spontaneous speech: Severely impaired, often limited to stereotyped utterances
- Auditory comprehension: Severely impaired
- Repetition: Severely impaired
- Naming: Severely impaired
- Reading: Severely impaired
- Writing: Severely impaired

Typical errors: Minimal meaningful verbal output, severe impairment across all language domains

Associated brain region: Extensive damage to perisylvian language areas

Case example: A 75-year-old man presents with almost no meaningful speech output following a large left hemisphere stroke. He is unable to follow commands, name objects, or engage in any form of written language.

Anomic Aphasia:
- Spontaneous speech: Fluent but with word-finding pauses
- Auditory comprehension: Preserved
- Repetition: Relatively preserved
- Naming: Significantly impaired
- Reading: Often preserved
- Writing: Mild to moderate impairment, mainly in word retrieval

Typical errors: Frequent word-finding difficulties, circumlocutions

Associated brain region: Various, often involving the angular gyrus or temporal-parietal junction

Case example: A 58-year-old woman presents with fluent speech but frequent pauses as she struggles to retrieve specific words, especially nouns. Her comprehension and repetition abilities are intact.

Transcortical Sensory Aphasia:
- Spontaneous speech: Fluent but often empty or irrelevant
- Auditory comprehension: Impaired
- Repetition: Preserved (key distinguishing feature)
- Naming: Impaired
- Reading: Impaired comprehension
- Writing: Impaired

Typical errors: Poor comprehension with preserved repetition, semantic paraphasias

Associated brain region: Watershed area between middle cerebral and posterior cerebral arteries

Case example: A 68-year-old man presents with fluent but often irrelevant speech. He has difficulty understanding complex commands but can repeat even long sentences accurately.

Transcortical Motor Aphasia:
- Spontaneous speech: Non-fluent, reduced output
- Auditory comprehension: Relatively preserved
- Repetition: Preserved
- Naming: Impaired
- Reading: Often preserved for comprehension
- Writing: Impaired

Typical errors: Reduced speech initiation, preserved repetition

Associated brain region: Frontal lobe, anterior or superior to Broca's area

Case example: A 72-year-old woman presents with reduced speech output and difficulty initiating conversation. However, she can repeat phrases and sentences without difficulty and shows good comprehension.
1. Fluency vs. Non-fluency
   - Fluent: Wernicke's, Conduction, Anomic, Transcortical Sensory
   - Non-fluent: Broca's, Global, Transcortical Motor

2. Comprehension
   - Good: Broca's, Conduction, Anomic, Transcortical Motor
   - Poor: Wernicke's, Global, Transcortical Sensory

3. Repetition
   - Impaired: Broca's, Wernicke's, Conduction, Global
   - Preserved: Anomic, Transcortical Sensory, Transcortical Motor

4. Naming
   - Severely Impaired: Wernicke's, Global, Anomic
   - Moderately Impaired: Broca's, Conduction, Transcortical types

Every type of aphasia is equally common and it differs on a patient by patient basis.
Provide your classification and a detailed justification based on the patient's performance across all domains.
"""


# Format the answer
answer_info = f"""
Transcript: {cleaned_content}. Look only at the PAR lines for this. compare the entire transcript to every line of the spreadsheet. dont compere teh words, but compare the speech patterns and the symprtoms of speech. the words are the same for all of them.


Can you analyze the entire spreadsheet. And every row
Compare the transcript to the spreadsheet. Give me the top 3 rows that you think align with this patient in transcript. Look only at the PAR lines for this
Look only at the PAR lines for this. Each row has language, gender, aphasia type, WAB Score, Transcript. Compare the symptoms of the transcript to the last column of each row in this file to find which symptoms match the spreadsheet. Then, look at the other information.
List:
Look only at the PAR lines for this. I want you to compare the entire sections. Look towards the ending of the transcript in fact. there is more content there
(row  in the spreadsheet; Display the first 3 lines and the line that you think connects the transcript the best)
(row  in the spreadsheet; Display the first 3 lines and the line that you think connects the transcript the best)
(row  in the spreadsheet; Display the first 3 lines and the line that you think connects the transcript the best)

Symptoms:
You seem to be confusing word retreival issues and paraphasias/neologisms/Comprehension issues. Try to fixt this


1. Fluency of speech: Is the speech effortful and non-fluent (like in Broca's aphasia) or more fluent but with errors (like in conduction or Wernicke's aphasia)? (number of occurrences), severity percentile (numerical)


2. Comprehension abilities: How well does the patient understand spoken language? This helps differentiate receptive (e.g. Wernicke's) from expressive (e.g. Broca's) aphasias.(number of occurrences), severity percentile (numerical)


3. Repetition skills: Ability to repeat words and phrases is a key diagnostic feature, particularly impaired in conduction aphasia.(number of occurrences), severity percentile (numerical)


4. Naming abilities: Word-finding difficulties are common in many aphasias, but the nature and severity can vary.(number of occurrences), severity percentile (numerical)


5. Presence and types of paraphasias: Phonemic paraphasias (sound errors) vs. semantic paraphasias (meaning-related errors) can point to different aphasia types.(number of occurrences), severity percentile (numerical)


6. Awareness of errors: Whether the patient recognizes and attempts to correct their errors.(number of occurrences), severity percentile (numerical)


7. Agrammatism: The degree to which grammatical structure is impaired.(number of occurrences), severity percentile (numerical)


8. Automatic speech: Ability to produce overlearned phrases or sequences.(number of occurrences), severity percentile (numerical)


9. Reading and writing abilities: These can be differentially affected in various aphasia types.(number of occurrences), severity percentile (numerical)


10. Overall pattern of strengths and weaknesses across language domains.(number of occurrences), severity percentile (numerical)

For accurate diagnosis, it's crucial to consider the full pattern of deficits and preserved abilities across all these areas, rather than relying too heavily on any single feature. A comprehensive language assessment considering all these factors is essential for proper aphasia classification.

Symptom Severity Rating:
Rate the following factors on a scale of 0-10:

Fluency: (insert score here; scale of 0-10)
Comprehension: (insert score here; scale of 0-10)
Repetition: (insert score here; scale of 0-10)
Naming: (insert score here; scale of 0-10)

Global Aphasia

Fluency: 0-5
Comprehension: 0-3.9
Repetition: 0-4.9
Naming: 0-6


Broca's Aphasia

Fluency: 0-9
Comprehension: 5-10
Repetition: 0-6
Naming: 0-8

Wernicke's Aphasia

Fluency: 0-3
Comprehension: 0-9
Repetition: 3-10
Naming: 0-6


Transcortical Motor Aphasia

Fluency: 0-5
Comprehension: 4-10
Repetition: 8-10
Naming: 0-8


Transcortical Sensory Aphasia

Fluency: 5-10
Comprehension: 0-6.9
Repetition: 8-10
Naming: 0-9


Mixed Transcortical Aphasia

Fluency: 5-10
Comprehension: 0-6.9
Repetition: 8-10
Naming: 0-9


Conduction Aphasia

Fluency: 5-10
Comprehension: 7-10
Repetition: 0-8
Naming: 0-9

Anomic Aphasia

Fluency: 7-10
Comprehension: 8-10
Repetition: 8-10
Naming: 0-9

Aphasia Classification:
1. Is speech fluent?
A fluency disorder is an interruption in the flow of speaking characterized by atypical rate, rhythm, and disfluencies (e.g., repetitions of sounds, syllables, words, and phrases; sound prolongations; and blocks), which may also be accompanied by excessive tension, speaking avoidance, struggle behaviors, and secondary mannerisms (American Speech-Language-Hearing Association [ASHA], 1993). People with fluency disorders also frequently experience psychological, emotional, social, and functional impacts as a result of their communication disorder (Tichenor & Yaruss, 2019a).
Make sure that you use the timestamps at the end of the transcripts to do this.
   Yes -> Check comprehension
   No -> Check comprehension for non-fluent types

2. Fluent types:
   - Poor comprehension, poor repetition -> Wernicke's
   - Poor comprehension, good repetition -> Transcortical Sensory
   - Good comprehension, poor repetition -> Conduction
   - Good comprehension, good repetition -> Anomic

3. Non-fluent types:
   - Poor comprehension -> Global
   - Good comprehension, poor repetition -> Broca's
   - Good comprehension, good repetition -> Transcortical Motor

Aphasia Classification Trial: (insert type of aphasia)

Provide your classification with detailed justification based on performance across all domains.
If all of the symptoms are less than 7 occurances, there is no aphasia
It is important to look at not just the symptoms but thte content of the speech too. Is what the patient is saying make sense? Do they loose track of the topic? Are they frustrated?
Initial Aphasia Diagnosis:
Provide 4 types of aphasia it could be. Transcortical Motor Aphasia. Don't forget about  Wernicke's aphasia orGlobal aphasia or Transcortical Motor Aphasia or Transcortical Sensory Aphasia
Initial Aphasia Types. For each of these types, compare back to the spreadsheet and check this transcript agaiinst every one of the transcripts for that type of aphasia. I dont
Wernicke's Aphasia
(insert type of aphasia or no aphasia)
(insert type of aphasia or no aphasia)
Considerations for Alternate Aphasias:
List reasons for considering each alternate aphasia. Include detailed comparisons and rationale.
Reevaluate the initial diagnosis if substantial evidence supports an alternate type.
Type of aphasia: Present this in the format of "Type of Aphasia: (insert aphasia type here)"

Provide a confidence score as well. If confidence is below 90% it is another type of aphasia.
Keep trying to figure it out.

Compare the transcript to the spreadsheet to determine aphasia type and WAB score.

WAB Score Determination:

Spontaneous Speech:
Content
Fluency


Auditory Verbal Comprehension:
Yes/No Questions
Auditory Word Recognition


Sequential Commands
Repetition:
Words
Phrases
Sentences


Naming and Word Finding:
Object Naming
Word Fluency
Sentence Completion
Responsive Speech


Aphasia Quotient (AQ): Overall score indicating the severity of aphasia.


1. Start at WAB score of 5; not 50 because wab can be lower than 50
2. Increase in 10-point intervals, comparing to the spreadsheet. Go all the way to 100
3. Provide reasoning for each step
4. When a suitable interval is found after going all the way to 100, refine by 5-point, then 1-point increments, then 0.1 point increments.
5. Remember: WAB scores can be below 60 or above 93.8 (no aphasia)
6. Determine a score based on the the following studies: The WAB has four subdomains: (1) spontaneous speech; (2) auditory comprehension; (3) repetition; and (4) naming designed to determine the presence of aphasia and judge the type of linguistic deficit and measure the severity of language impairment. The full score of spontaneous speech is 20, including fluency and content of information. The 200 points of auditory comprehension consist of 60 points of Yes–No questions, 60 points of auditory word recognition, and 80 points of sequential commands. The total score of repetition is 100. A total of 100 points are given for naming, including 60 points for object naming, 20 points for fluency of words, 10 points for sentence completion, and 10 points for responsive speech. The calculation formula of the aphasia quotient is “AQ = (Spontaneous + Comprehension ÷ 20 + Repetition ÷ 10 + Naming ÷ 10) × 2”. All of the items result in a range of possible scores from 0 to 100. Combined with the clinical data of stroke patients, those whose aphasia quotient is less than 93.8 can be judged as aphasia, and the smaller the aphasia quotient, the more serious the aphasia. In addition, the WAB-AQ is commonly used to measure recovery, although there is a discrepancy between clinical impression and WAB in the classification of aphasia, the superiority of WAB is that it can quantify language damage.
7. Determine a score by comparing this patient with the participants in the spreadsheet.
8. Compare the 3 scores and determine a final score.

Show your thoughts below:
WAB Score 1:
WAB Score 2:
WAB Score 3:

WAB Score: Present this in the format of "Estimated WAB Score: (insert number on scale of 0-100 here)". Note that WAB scores can range widely and are not limited to 50-80.
IMPORTANT: WAB SCORE IS NOT RELATED TO TYPE OF APHASIA

Final Type of aphasia: Based on the WAB score and the original aphasia type, make any changes that you think would be mandatory. If you are borderline 93.8, check if the no aphasia option makes sense. Use the spreadsheet

Treatment Suggestions:
Provide treatment suggestions in a list, with a description for each therapy. Include a detailed explanation for choosing each treatment, citing the aphasia manual and patient symptoms.


Explanation:
(Provide a detailed explanation here for the therapist, ensuring it is useful and practical, citing the manual where appropriate).
"""


if uploaded_files:
    for uploaded_file in uploaded_files:
        st.write(f"Processing file: {uploaded_file.name}")
        file_content = uploaded_file.read().decode("utf-8")
        cleaned_content = clean_transcript(file_content)

        prompt = f"""
You are a speech pathologist, a healthcare professional who specializes in evaluating, diagnosing, and treating communication disorders, including speech, language, cognitive-communication, voice, swallowing, and fluency disorders. Your role is to help patients improve their speech and communication skills through various therapeutic techniques and exercises.

In this scenario, you will be working with a patient who has the following speech disorder or issue:

Aphasia

What is aphasia?
Aphasia is a disorder that results from damage to portions of the brain that are responsible for language. For most people, these areas are on the left side of the brain. Aphasia usually occurs suddenly, often following a stroke or head injury, but it may also develop slowly, as the result of a brain tumor or a progressive neurological disease. The disorder impairs the expression and understanding of language as well as reading and writing. Aphasia may co-occur with speech disorders, such as dysarthria or apraxia of speech, which also result from brain damage.

Who can acquire aphasia?
Most people who have aphasia are middle-aged or older, but anyone can acquire it, including young children. About 1 million people in the United States currently have aphasia, and nearly 180,000 Americans acquire it each year, according to the National Aphasia Association.

What causes aphasia?
Aphasia is caused by damage to one or more of the language areas of the brain. Most often, the cause of the brain injury is a stroke. A stroke occurs when a blood clot or a leaking or burst vessel cuts off blood flow to part of the brain. Brain cells die when they do not receive their normal supply of blood, which carries oxygen and important nutrients. Other causes of brain injury are severe blows to the head, brain tumors, gunshot wounds, brain infections, and progressive neurological disorders, such as Alzheimer's disease.

use the instructions to do the full analysis: {instructions}

cite specific content from the information below to justify your decisions

<dialogue>
Focus on the *PAR lines
Remember that WAB score is not related to Aphasia type. Every WAB score can be any aphasia and every aphasia can have a range of WAB scores
{cleaned_content}
</dialogue>

Based on the information provided, please do the following:

Focus on the *PAR lines
Do this using {answer_info}

Remember, your goal is to provide a comprehensive and supportive therapy experience for the patient, helping them to improve their speech and communication skills and build their confidence in the process. Use your expertise and empathy to guide them through this journey.

    """

        try:
            response = call_bedrock(prompt)
            st.write(response)

            # Add a pause to avoid rate limiting
            time.sleep(2)  # Adjust this value as needed

        except Exception as e:
            st.error(f"An error occurred while processing the file {uploaded_file.name}: {str(e)}")

else:
    st.info("Please upload files to analyze.")

ModuleNotFoundError: No module named 'streamlit'